### Setting parameter values

In [ ]:
!pip install pybamm
import pybamm

In [ ]:
# print the list of in-built parameter sets
parameter_values = pybamm.ParameterValues("Chen2020")
parameter_values

In [ ]:
# access specific parameter via dictionary index
parameter_values["Electrode height [m]"]

In [ ]:
# search parameter values
parameter_values.search("electrolyte")

In [ ]:
# running a simulation with a given parameter set by passing "parameter_values" as the keyword argument
model = pybamm.lithium_ion.DFN()
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sim.solve([0, 3600])
sim.plot()

### Change individual parameters
We often want to quickly change a small number of parameter values to investigate how the behaviour or the battery changes. In such cases, we can change parameter values directly in notebook or script we are working in as we demonstrate in this section.

Parameters can either have a constant value or be a function of a model variable. These dependencies are hardcoded into the PyBaMM models. In order to check what are the specific parameters that a model requires, and whether they must be constants or they can be functions, we can call the print_parameter_info method.

In [ ]:
model.print_parameter_info()

The table shows all the parameters that need to be provided to the model (in this case DFN). For example, we see that we need to define (amongst others) the Negative electrode thickness [m], which is defined to be a Parameter. This means it can only have a constant value. In contrast, other parameters are defined to be a FunctionParameter, which means that they can depend on model variables. For example, the Current function [A] is a FunctionParameter that can depend on Time [s].

Note that a FunctionParameter can always be defined to be a constant (i.e. like if it was a Parameter), but a Parameter cannot be defined to be a function. This is because these dependencies are hardcoded in the model definitions.

In [ ]:
# changing the current function to 10A
parameter_values["Current function[A]"] = 10
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sim.solve([0, 3600])    # solving the model from t=0 -> 3600 (1hr)
sim.plot()

In [ ]:
# changing the current function to a time-dependent function
import numpy as np

def my_current(t):      # sets the change in current to be sinusoidal 
    return pybamm.sin(2 * np.pi * t / 60)

parameter_values["Current function [A]"] = my_current   
# "my_current" function takes an argument "t" (time)
# the name given to the argument doesn't matter, but the order does. Eg: for "Electrolyte conductivity", it assumes the first is "Electrolyte concentration" and 
# the second is "Temperature"

In [ ]:
# to solve the model again
sim = pybamm.Simulation(model, parameter_values=parameter_values)
t_eval = np.arange(0, 121, 1)       # array of points we want to evaluate our solution to the solver
sim.solve(t_eval=t_eval)
sim.plot(["Current [A]", "Voltage [V]"])

### Parameter sweeps
If the value of a parameter is expected to change, it is better to set a parameter as an "input parameter".

In [ ]:
# Set the value of input parameter to the string "input" in the parameter value dictionary

import matplotlib.pyplot as plt

parameter_values["Current function [A]"] = "[input]"
sim = pybamm.Simulation(model, parameter_values=parameter_values)
solns = []
for c in [0.1, 0.2, 0.3]:
    soln = sim.solve([0, 3600], inputs={"Current function [A]": c})
    plt.plot(soln["Time [s]"].entries, soln["Voltage [V]"].entries, label=f"{c} A")
    solns.append(soln["Terminal voltage [V]"].entries)
plt.xlabel("Time [s]")
plt.ylabel("Terminal voltage [V]")
plt.legend()
plt.show()

In [ ]:
# To define a new parameter set (eg: for a new battery or chemistry to repeat simulations on):
# Initialise a "ParameterValues" object and pass as an argument to the parameter values dictionary

def cube(t):
    return t**3


parameter_values = pybamm.ParameterValues(
    {
        "Negative electrode thickness [m]": 1e-4,
        "Positive electrode thickness [m]": 1.2e-4,
        "Current function [A]": cube,
    }
)

Note how, when we pass a function as a parameter, we pass the object without calling it, i.e. we pass cube rather than cube(t). This new parameter_values variable could now be passed to a simulation, but note that it is incomplete as it does not include all the parameters that the model needs to run (see the parameters needed by calling model.print_parameter_info(), as done above).

It is often convenient to define the parameter set in a separate file, and then call the parameters into your notebook or script. You can find some examples on how to do so in PyBaMM’s parameter library. You can copy one of the parameter sets available into a new file and modify it accordingly for the new parameter set. Then, whenever the set is needed, one can import the get_parameter_values method from the corresponding file and call it to obtain a copy of the parameter values.